Look at the stats for just the undirdir runs to see how changing the random seed changes things.

In [1]:
import sys, os, time, json, re
from glob import glob

In [2]:
g = glob('../undirdir/job_logs/infomap_logs/*.out')

In [58]:
infomap_stats = {}  # keys are log filenames (random seed at the end)


In [6]:
tfname = '../undirdir/job_logs/infomap_logs/slurm-1160295_101.out'
with open(tfname, 'r') as f:
    ttxt = f.read()

In [45]:
patterns = {
    'perlevel_num_modules': re.compile(r"per level number of modules:.*?(\[.+?\])", re.I|re.M),
    'perlevel_num_leafnodes': re.compile(r"per level number of leaf nodes:.*?(\[.+?\])", re.I|re.M),
    'perlevel_avg_child_degree': re.compile(r"per level average child degree:.*?(\[.+?\])", re.I|re.M),
    'perlevel_codelength_modules': re.compile(r"per level codelength for modules:.*?(\[.+?\])", re.I|re.M),
    'perlevel_codelength_leafnodes': re.compile(r"per level codelength for leaf nodes:.*?(\[.+?\])", re.I|re.M),
    'perlevel_codelength_total': re.compile(r"per level codelength total:.*?(\[.+?\])", re.I|re.M),
}

In [46]:
m = patterns['perlevel_num_modules'].findall(ttxt)

In [47]:
m

['[      41137,        8141,       64962,      155415,       74391,       12587,        1076,          71,           0]']

In [49]:
if m:
    tj = json.loads(m[0])

In [51]:
tj

[41137, 8141, 64962, 155415, 74391, 12587, 1076, 71, 0]

In [59]:
%%time
for fname in g:
    fname_tail = os.path.split(fname)[1]
    infomap_stats[fname_tail] = {}
    with open(fname, 'r') as f:
        txt = f.read()
    for pname, p in patterns.items():
        m = p.findall(txt)
        if m:
            d = json.loads(m[0])
        else:
            d = None
        infomap_stats[fname_tail][pname] = d

CPU times: user 200 ms, sys: 21 ms, total: 221 ms
Wall time: 4.93 s


In [60]:
infomap_stats

{'slurm-1160295_100.out': {'perlevel_avg_child_degree': None,
  'perlevel_codelength_leafnodes': None,
  'perlevel_codelength_modules': None,
  'perlevel_codelength_total': None,
  'perlevel_num_leafnodes': None,
  'perlevel_num_modules': None},
 'slurm-1160295_101.out': {'perlevel_avg_child_degree': [41137,
   2.30639,
   10.3495,
   12.6935,
   5.45864,
   2.95108,
   2.38722,
   2.0316,
   2.01408],
  'perlevel_codelength_leafnodes': [0.0,
   0.001311257,
   0.010635538,
   3.504290221,
   1.909629098,
   0.132568356,
   0.009472178,
   0.000354896,
   1.6541e-05],
  'perlevel_codelength_modules': [0.017590942,
   0.801714758,
   3.402853418,
   1.480508236,
   0.075535659,
   0.003552867,
   0.000124545,
   4.611e-06,
   0.0],
  'perlevel_codelength_total': [0.017590942,
   0.803026015,
   3.413488956,
   4.984798457,
   1.985164757,
   0.136121223,
   0.009596723,
   0.000359506,
   1.6541e-05],
  'perlevel_num_leafnodes': [0,
   86737,
   19293,
   669181,
   773963,
   206947,
 

In [ ]:
import cPickle as pickle
with open('undirdir_log_stats.pickle', 'wb') as outf:
    